<a href="https://colab.research.google.com/github/Parfy/colab-music-feature-extraction/blob/master/Music_Feature_Extraction_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install essentia

In [0]:
#Imports
from google.colab import files
import essentia
import essentia.standard as es
from essentia.standard import *
import numpy as np

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Major_Lazer_-_Light_It_Up.mp3 to Major_Lazer_-_Light_It_Up.mp3
User uploaded file "Major_Lazer_-_Light_It_Up.mp3" with length 2659098 bytes


In [0]:
file_name = next(iter(uploaded)) #Assuming I've only uploaded one track 

In [0]:
!mkdir assets

In [0]:
# Loading audio file
audio = MonoLoader(filename=file_name)()

# Compute beat positions and BPM
rhythm_extractor = RhythmExtractor2013(method="multifeature")
bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)

#Save beat timings to text file
np.savetxt('assets/beats.txt', beats)

# Mark beat positions on the audio and write it to a file
# Let's use beeps instead of white noise to mark them, as it's more distinctive
marker = AudioOnsetsMarker(onsets=beats, type='beep')
marked_audio = marker(audio)

debug_file = 'assets/' + file_name + '_beats_debug.mp3'

MonoWriter(filename=debug_file)(marked_audio)

In [0]:
# Load audio file; it is recommended to apply equal-loudness filter for PredominantPitchMelodia
loader = EqloudLoader(filename=file_name, sampleRate=44100)
audio = loader()
# Extract the pitch curve
# PitchMelodia takes the entire audio signal as input (no frame-wise processing is required)

pitch_extractor = PredominantPitchMelodia(frameSize=2048, hopSize=128)
pitch_values, pitch_confidence = pitch_extractor(audio)

# Pitch is estimated on frames. Compute frame time positions
pitch_times = numpy.linspace(0.0,len(audio)/44100.0,len(pitch_values) )

# Put pitch and times into one array, save as text file ready for download
pitch = np.column_stack((pitch_times, pitch_values))
np.savetxt('assets/pitch.txt', pitch, delimiter=',')

In [0]:
#Zip required file together so they can be downloaded as one
!zip -r assets.zip assets
files.download('assets.zip')

  adding: assets/ (stored 0%)
  adding: assets/beats.txt (deflated 63%)
  adding: assets/Major_Lazer_-_Light_It_Up.mp3_beats_debug.mp3 (deflated 7%)
  adding: assets/pitch.txt (deflated 78%)
